In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import uuid
import tensorflow_datasets as tfds
from tensorflow.train import BytesList,Int64List,FloatList
from tensorflow.train import Example,Features,Feature

In [ ]:
data=tfds.load(name="malaria")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteMW16NZ/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
data=data["train"].shuffle(buffer_size=20)

In [ ]:
os.makedirs("malaria",exist_ok=True)
os.makedirs(os.path.join("malaria","0"),exist_ok=True)
os.makedirs(os.path.join("malaria","1"),exist_ok=True)
for img_lab in data.take(500):
      cv2.imwrite(os.path.join("malaria","1",f"{str(uuid.uuid1())}.jpg"),img_lab["image"].numpy()) if img_lab["label"].numpy() else cv2.imwrite(os.path.join("malaria","0",f"{str(uuid.uuid1())}.jpg"),img_lab["image"].numpy())


In [ ]:
train_data,val_data=tf.keras.utils.image_dataset_from_directory(os.path.join("malaria"),subset="both",validation_split=0.2,seed=12)

Found 500 files belonging to 2 classes.
Using 400 files for training.
Using 100 files for validation.


In [ ]:
# !rm -rf "malaria"

In [ ]:
def to_return_Example(img,lab):

  bytes_record=Feature(
      bytes_list=BytesList(value=[img])
  )

  int_record=Feature(
      int64_list=Int64List(value=[lab])
  )

  example=Example(
      features=Features(feature=
         { "images":bytes_record,
          "labels":int_record}
      )
  )

  return example.SerializeToString()

In [ ]:
def encoded_img(img,lab):
  img=tf.image.convert_image_dtype(img,dtype=tf.uint8)
  encoded_img=tf.io.encode_jpeg(img)
  return encoded_img,lab

In [ ]:
train_data=train_data.unbatch()

In [ ]:
val_data=val_data.unbatch()

In [ ]:
train_data_encode=train_data.map(encoded_img)
val_data_encode=val_data.map(encoded_img)

In [ ]:
NUM_SHORDS=10
for shard_number in range(10):
  shard_data=train_data_encode.shard(NUM_SHORDS,shard_number).as_numpy_iterator()
  with tf.io.TFRecordWriter(f'{os.path.join("tfrecords",f"{shard_number}.tfrecord")}') as filewriter:
    for img,lab in shard_data:
      filewriter.write(to_return_Example(img,lab))

In [ ]:
recon_dataset=tf.data.TFRecordDataset(filenames=[os.path.join("tfrecords",p)for p in os.listdir("tfrecords")])


In [ ]:
def convert_tfrecord_tfdataset(example):
  feature_description={
      "images":tf.io.FixedLenFeature([],tf.string),
      "labels":tf.io.FixedLenFeature([],tf.int64)
  }
  example=tf.io.parse_single_example(example,feature_description)
  example["images"]=tf.io.decode_jpeg(example["images"],channels=3,dtype=tf.float32)
  return example["images"],example["labels"]

In [ ]:
def parse_example(recon_dataset):
  feature_description={
        "images":tf.io.FixedLenFeature([],tf.string),
        "labels":tf.io.FixedLenFeature([],tf.int64)
    }
  return tf.io.parse_single_example(recon_dataset,feature_description)
# convert_tfrecord_tfdataset(recon_dataset)

In [ ]:
example=recon_dataset.map(parse_example)

In [ ]:
def decode_image(img_lab):
  return tf.io.decode_jpeg(img_lab["images"],channels=3),img_lab["labels"]
tf_data=example.map(decode_image)

In [ ]:
list(tf_data.take(1))

[(<tf.Tensor: shape=(256, 256, 3), dtype=uint8, numpy=
  array([[[  0, 255,   3],
          [  0, 255,   3],
          [  3, 254,   1],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0, 255,   1],
          [  0, 255,   1],
          [  3, 254,   1],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0, 255,   0],
          [  0, 255,   0],
          [  1, 255,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         ...,
  
         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0],
          ...,
          [  0,   0,   0],
          [  0,   0,   0],
          [  0,   0,   0]],
  
         [[  0,   0,   